# Chapter 7. 질문에 답하기  

질의응답이란 질문에 답을 하는 과제입니다.  
이 장에서는 LG CNS가 공개한 KorQuAD 1.0 데이터셋을 가지고 질의응답 모델을 구축하는 방법을 살펴봅니다.

## <7-1 질의응답 모델 훑어보기>   

이 장에서는 질의응답 과제를 실습하면서 모델의 아키텍처, 입출력 등 전반을 살펴보겠습니다.  
***질의응답(question answering)*** 이란 질문에 답을 하는 과제입니다. 질의응답 과제의 유형은 다양하지만 이 책의 실습에서는 다음 예시처럼 질문에 대한 답을 지문(context)에서 찾는 것입니다.

- **지문(context)**: 한글은 홀소리와 닿소리 모두 소리틀을 본떠 만든 음소문자로 한글 맞춤법에서는 닿소리 14개와 홀소리 10개, 모두 24개를 표준으로 삼는다. "나랏말이 중국과 달라" 문제를 느낀 조선의 세종대왕이 한국어는 물론 이웃 나랏말까지 나타내도록 1443년 창제하여 1446년 반포하였다.  
- **질문(question)**: 한글이 창제된 연도는?  
- **답변(answer)**: 1443년

위와 같은 형태의 질의응답 과제에서 모델의 입력은 질문과 지문, 출력은 입력의 각 토큰이 [정답의 시작일 확률, 정답의 끝일 확률]인 형태입니다.  
예를 들면 다음과 같습니다.

> (입력) :  
한글이 창제된 연도는? 한글은 홀소리와 닿소리 모두 소리틀을 본떠 만든 음소문자로 한글 맞춤법에서는 닿소리 14개와 홀소리 10개, 모두 24개를 표준으로 삼는다. “나랏말이 중국과 달라” 문제를 느낀 조선의 세종대왕이 한국어는 물론 이웃나라 말까지 나타내도록 1443년 창제하여 1446년 반포하였다.  

> (출력) :  
(…생략…)  
나타내도록 → [0.02, 0.01]   
`1443` → [`0.93`, 0.01]    
`년` → [0.01, `0.90`]    
창제하여 → [0.01, 0.01]      
(…생략…)

우선 `나타내도록`을 봅시다.   
여기에서 '정답의 시작일 확률(0.02)'과 '정답의 끝일 확률(0.01)'의 합이 1이 안 되는 것에 의문을 가질 수도 있을 것 같습니다. 정답의 시작일 확률과 끝일 확률은 각각 전체 토큰에 걸쳐 계산됩니다. 다시 말해 정답의 시작과 관련된 확률 분포는 […, 0.02, 0.93, 0.01, …]이며 […, `나타내도록`, `1443`, `년`, `창제하여`, …]에 각각 대응됩니다. 마찬가지로 정답의 끝과 관련된 확률 분포는 […, 0.01, 0.01, 0.90, 0.01, …]이며 […, `나타내도록`, `1443`, `년`, `창제하여`, …]에 각각 대응됩니다.

질의응답 모델의 이러한 출력 확률을 적당한 후처리 과정을 거쳐 사람이 보기에 좋은 형태로 가공해 줍니다. 정답의 시작 확률 분포에서 가장 큰 확률값을 가진 토큰부터, 정답의 끝 확률 분포에서 가장 큰 확률값을 가진 토큰까지 이어붙이면 됩니다. 앞의 예시에서는 `1443 + 년`이 모델이 예측하는 정답이 됩니다.

이 책의 질의응답 실습에서는 LG CNS가 공개한 `KorQuAD 1.0` 데이터를 활용해 모델을 구축하는 실습을 진행합니다.  

cf) korquad.github.io/KorQuad%201.0/

### 모델 구조
---
이 책에서 사용하는 질의응답 모델은 다음 그림과 같은 구조입니다.   
질문과 지문을 각각 토큰화한 뒤 `'[CLS] 질문 [SEP] 지문 [SEP]'` 형태로 이어 붙입니다. 여기에서 `CLS`는 문장 시작을 알리는 스페셜 토큰, `SEP`는 질문과 지문을 서로 구분해 주는 의미의 스페셜 토큰입니다. 이를 BERT 모델에 입력하고 모든 입력 토큰에 대해 BERT 모델 마지막 레이어의 출력을 뽑습니다. 이들 토큰 벡터 각각에 작은 추가 모듈을 덧붙여 모델의 출력이 [해당 지문 토큰이 정답의 시작일 확률, 해당 지문 토큰이 정답의 끝일 확률]이 되도록 합니다.

<center><질의응답 모델></center>

<p align="center"><img src="https://i.imgur.com/b5qXRYa.png">  



### 태스크 모듈
---
질의응답 모델에 붙는 모듈의 구조는 다음 그림과 같습니다.  
우선 마지막 레이어의 개별 토큰 벡터 각각(그림에서 $x$)에 드롭아웃을 적용합니다. 그다음 가중치 행렬을 곱해 2차원 벡터로 변환합니다(그림에서 $h$). 개별 토큰 벡터 각각을 2차원으로 사영(projection)하는 이유는 각각이 정답의 시작이냐 아니냐, 정답의 끝이냐 아니냐 정보를 나타내기 때문입니다.  
만일 마지막 레이어의 개별 토큰 벡터가 768차원이라면 가중치 행렬 크기는 768 $\times$ 2가 됩니다.

<center><질의응답 태스크 모듈></center>

<p align="center"><img src="https://i.imgur.com/HvDoJVa.png">   



<center>출처 : ratsgo</center>



이후 $h$의 첫 번째 값들만 모아서 소프트맥스를 취하면 정답의 시작과 관련한 모델의 출력 $y$가 됩니다. $h$의 두 번째 값들만 모아서 소프트맥스를 취하면 정답의 끝과 관련한 모델의 출력 $z$입니다. 만일 입력 토큰 개수가 $n$개라면 $y,z$ 모두 그 차원 수는 $n$이 됩니다.

이 실습에서 사용되는 레이블은 입력 토큰 기준 정답의 시작 위치와 끝 위치입니다. 위와 같이 만든 모델의 최종 출력($y, z$)과 정답 레이블(토큰 기준 위치)을 비교해 모델 출력이 정답 레이블과 최대한 같아지도록 BERT 레이어를 포함한 모델 전체를 업데이트합니다.  
이를 파인튜닝이라고 합니다.